<h1>Week 3 Assignment - Bautifulsoup Web scraping

<h3>Step 1: Import Libraries needed, scrap data and diplay as pandas dataframe
    

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

x= ('postalcode', 'Borough', 'Neighborhood')

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table), header=0)[0]

df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


<h3> Step 2: Drop rows where Borough is 'Not assigned'

In [2]:
df2 = df[df.Borough != 'Not assigned']

df2.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<h3> Step 3: Check which Neighbourhoods are listed as 'Not assigned'

In [3]:
df2[df2["Neighbourhood"].str.contains("Not assigned")]

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


<h3> Step 4: Change the 'Not assinged' from Queens Park to 'Queen's Park' 

In [4]:
 df2.replace('Not assigned', 'Queen\'s Park').head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


<h3> Step 5: Group by Postcode and Bourough and separte Neighbourghoods by commas.

In [5]:
df3 = df2.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()

# df3.reset_index(level=1, inplace=True)
df3.head()
# df3.reset_index(1)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h3> Step 6: Print the Shape of the Final DataFrame
   

In [6]:
d = df3.shape

print("The shape of df3 is : Rows->", d, '<-Columns')

The shape of df3 is : Rows-> (103, 3) <-Columns


<h3> Step 7: Geocoder Python package DID NOT WORK.... moving on to CSV file

In [7]:
# !pip install geocoder


In [8]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('{}, Toronto, Ontario'.format('Postcode'))
#   lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

<h3> Step 8:  Combine CSV file to Create df4 Dataframe 

In [9]:
import pandas as pd

In [10]:
!wget -q -O 'postal_code_TO.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [11]:
Postal_Lat_Longdf = pd.read_csv('postal_code_TO.csv')
Postal_Lat_Longdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df4 = df3.merge(Postal_Lat_Longdf, left_on='Postcode', right_on='Postal Code', how='outer')
    
df4 = df4.drop(['Postal Code'], axis=1)

df4.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
